In [1]:
%%bash
chia version
cdv --version

1.3.5
cdv, version 1.0.8


# What Is Singleton?

> **A singleton is an identity which is embodied by exactly one coin** at any given time, and where it’s possible for coins to validate that they’re interacting with the current representative of that identity. It’s similar to, but simpler than, coloured coins. An NFT is barely more than a raw singleton. Singletons are also the foundation of rate limited wallets, distributed identities, pool protocol, and price oracles. -- [A Vision for DeFi in Chia](https://www.chia.net/2021/07/13/a-vision-for-defi-in-chia.en.html)

> **A singleton is a coin that is verifiably unique**. Similar to (but more powerful than) NFTs, a singleton cannot be copied, duplicated, or recreated if it is destroyed. It is a common outer puzzle for things like DIDs, NFTs, or the pooling protocol. -- [The Great Chia Glossary | Singleton](https://chialisp.com/docs/glossary)

> **A singleton is a special kind of puzzle that asserts uniqueness**, preventing other coins from being mistaken for it. It can change state over time, each time creating a new puzzle based on the previous. It wraps an `inner puzzle` which can have whatever behavior you desire. These traits combined allow for smart contracts and things like NFTs to be created. -- [What is a singleton?](https://developers.chia.net/t/what-is-a-singleton/87)

## Why Do We Need Singleton?
> Coins on the network are state, the puzzle is the data contained within them. You can spend them to change their state however you like, by creating new coins. This is what singletons excel at. -- [Can you store state on the network?](https://developers.chia.net/t/can-you-store-state-on-the-network/84)

In the account model (e.g, Ethereum), you have **a unique contract address** that you can access and update data. In the coin set model, however, coins are state, so we need to be able to extract state that we are interested from those coins somehow. To update those states, we will need to spend them and create new coins. 

### Coin Id & Puzzle Hash
Each coin has a unique [coin_id](../../basic/coins/notebook.ipynb) which derived from `parent_coin_id`, `puzzle_hash`, and its `amount` (in mojos). 

Once you spend the coin with the existing state, you could get a new unspent coin with the new state (different or same puzzle hash). However, this means that you have to keep track of a valid `coin_id` and/or `puzzle hash`. In addition, someone can just create new coins with the same puzzle hash out of nowhere which could be more confusing.

### A Singleton Puzzle & Launcher Id
It's clear that we need a way to model the idea of **unique id**, so the Chia Network comes up with [singleton puzzle](https://chialisp.com/docs/puzzles/singletons). **A singleton coin provides a unique id called launcher id that can be used as the permanent unique address.** The `launcher id` is basically the `coin_id` of the launcher coin that creates the first singleton coin. When a singleton coin is spent, at most one new singleton coin can be created. We can always track all of those singleton coins and find the unspent singleton coin by following the `launcher id`.


## Singleton Puzzle

The singleton puzzle is an outer puzzle that can wrap any inner puzzle. This ensures that all singleton coins follow the rules of singleton.

### Rules of Singleton
- A singleton must have an odd mojo value.
- Only one output (i.e., a singleton) coin can have an odd mojo value.
- All singletons should have the same format.

### Boot Strapping

To create **the first singleton coin (eve)**, we spend a standard coin and a [singleton launcher coin](https://github.com/Chia-Network/chia-blockchain/blob/1.3.5/chia/wallet/puzzles/singleton_launcher.clvm).

- The first singleton coin (eve) must be created from the coin called the Launcher by passing in our `singleton_puzzle_hash` (as a solution).
- `singleton_puzzle_hash` is a [singleton top layer puzzle](https://github.com/Chia-Network/chia-blockchain/blob/1.3.5/chia/wallet/puzzles/singleton_top_layer.clvm) curried in `SINGLETON_STRUCT` and `INNER_PUZZLE`.
- `SINGLETON_STRUCT` is `(SINGLETON_MOD_HASH . (LAUNCHER_ID . SINGLETON_LAUNCHER_HASH))`.
- `SINGLETON_MOD_HASH` is a puzzle hash of the [singleton top layer puzzle](https://github.com/Chia-Network/chia-blockchain/blob/1.3.5/chia/wallet/puzzles/singleton_top_layer.clvm).
- To make sure that the `singleton_puzzle_hash` is not tampered, we will utilize annoucement which contains `singleton_puzzle_hash`, `odd_amount`, and list of keys/values .
- The starting coin (which can be a standard transaction coin) creates the launcher coin and assert the announcement from the same launcher coin that is spent at the same time. The launcher coin is an **ephemeral** coin.

<img src="bootstrapping_singleton.jpg" alt="Boot Strapping Singleton" width="600"/>


## Code
### Singleton Top Layer & Singleton Launcher

- [singleton_top_layer.py](https://github.com/Chia-Network/chia-blockchain/blob/1.3.5/chia/wallet/puzzles/singleton_top_layer.py)
- [singleton_top_layer.clvm](https://github.com/Chia-Network/chia-blockchain/blob/1.3.5/chia/wallet/puzzles/singleton_top_layer.clvm)
- [singleton_launcher.clvm](https://github.com/Chia-Network/chia-blockchain/blob/1.3.5/chia/wallet/puzzles/singleton_launcher.clvm)

In [2]:
from chia.wallet.puzzles import (singleton_top_layer, p2_delegated_puzzle_or_hidden_puzzle)

print(f'SINGLETON_MOD_HASH:\t\t{singleton_top_layer.SINGLETON_MOD_HASH}')
print(f'SINGLETON_LAUNCHER_HASH:\t{singleton_top_layer.SINGLETON_LAUNCHER_HASH}')

SINGLETON_MOD_HASH:		24e044101e57b3d8c908b8a38ad57848afd29d3eecc439dba45f4412df4954fd
SINGLETON_LAUNCHER_HASH:	eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9


### Starting Coin
First, let's prepare the starting coin. In this notebook, we will just use the blockchain simulator (and cdv.test).

In [3]:
# starting coin
import json
def print_json(dict):
    print(json.dumps(dict, sort_keys=True, indent=4))

from cdv.test import Network, Wallet

network: Network = await Network.create()
await network.farm_block()

alice: Wallet = network.make_wallet("alice")
await network.farm_block(farmer=alice) # alice has 2000_000_000_000 mojos

print(f'alice: {alice.balance()}')

odd_amount = 1023 # odd mojo
starting_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(alice.pk()) # Pre-commit DEFAULT_HIDDEN_PUZZLE_HASH
coin_records = await network.sim_client.get_coin_records_by_puzzle_hash(starting_puzzle.get_tree_hash())
starting_coin = next(cr.coin for cr in coin_records if cr.spent == False and cr.coin.amount >= 1_750_000_000_000)
print(starting_coin.name())
print_json(starting_coin.to_json_dict())


alice: 2000000000000
12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba
{
    "amount": 1750000000000,
    "parent_coin_info": "0xe3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001",
    "puzzle_hash": "0x4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3"
}


### Launcher Coin
Although the launcher coin is not on the blockchain yet, we can still create a reference to it by providing, the `parent_coin_info`, `puzzle_hash`, and the `amount`. The launcher coin will be created by the starting coin, so we use its `coin_id` as `parent_coin_info`. `puzzle_hash` is the `SINGLETON_LAUNCHER_HASH` and the amount is the odd amount that we want our singleton coin to have.

In [4]:
from chia.types.blockchain_format.coin import Coin

# or singleton_top_layer.generate_launcher_coin()
launcher_coin = Coin(starting_coin.name(), singleton_top_layer.SINGLETON_LAUNCHER_HASH, odd_amount)
launcher_id = launcher_coin.name()
print(launcher_id)
print_json(launcher_coin.to_json_dict())

6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570
{
    "amount": 1023,
    "parent_coin_info": "0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba",
    "puzzle_hash": "0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9"
}


### Singleton Puzzle
Since Chia Network already provide the [reference singleton puzzle](https://github.com/Chia-Network/chia-blockchain/blob/1.3.5/chia/wallet/puzzles/singleton_top_layer.clvm), we will use it as an outer puzzle and we will provide our inner puzzle which in this example, the standard coin puzzle (same puzzle as the starting coin) adapted to be used as an inner puzzle for singleton.

We curry in `SINGLETON_STRUCT` and the adapted inner puzzle to the reference singleton puzzle and that puzzle will be the puzzle of our first singleton coin.

In [5]:
# Singleton Puzzle
# The inner puzzle is the standart transaction puzzle (but can be any puzzle)
inner_puzzle = singleton_top_layer.adapt_inner_to_singleton(starting_puzzle)
print(inner_puzzle.get_tree_hash())
singleton_struct = (
    singleton_top_layer.SINGLETON_MOD_HASH, 
    (launcher_id, singleton_top_layer.SINGLETON_LAUNCHER_HASH)
)
curried_singleton_puzzle = singleton_top_layer.SINGLETON_MOD.curry(
            singleton_struct,
            inner_puzzle,
        )
singleton_puzzle_hash = curried_singleton_puzzle.get_tree_hash()
print(singleton_puzzle_hash)

206318efd67658a08ae16781f0819a216437a7774d98d2af101f8f8d7a03a02e
a1af73b7dc0f246ffb0bd41b4ac8f0253f4c0949bfdd070d52a3be037767baa3


### Lanucher Solution & Coin Spend
Let's look at the launcher coin again. To create our singleton coin, we will spend the launcher coin (at the same time it's created by the starting coin!).

The solution for the [launcher coin puzzle]((https://github.com/Chia-Network/chia-blockchain/blob/1.3.5/chia/wallet/puzzles/singleton_launcher.clvm) ) is the followings:

`(singleton_full_puzzle_hash amount key_value_list)`

Both `singleton_puzzle_hash` and `amount` are required while `key_value_list` is the extra parameter that we can used to inform any observer that our singleton has been created.

In [6]:
from chia.types.coin_spend import CoinSpend
from chia.types.blockchain_format.program import Program
from clvm.casts import int_to_bytes

# Launcher Solution
launcher_solution = Program.to(
    [
        singleton_puzzle_hash,
        int_to_bytes(odd_amount),
        [("Key", "Value")],
    ]
)

launcher_coin_spend = CoinSpend(
        launcher_coin,
        singleton_top_layer.SINGLETON_LAUNCHER,
        launcher_solution
)
print_json(launcher_coin_spend.to_json_dict())

{
    "coin": {
        "amount": 1023,
        "parent_coin_info": "0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba",
        "puzzle_hash": "0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9"
    },
    "puzzle_reveal": "0xff02ffff01ff04ffff04ff04ffff04ff05ffff04ff0bff80808080ffff04ffff04ff0affff04ffff02ff0effff04ff02ffff04ffff04ff05ffff04ff0bffff04ff17ff80808080ff80808080ff808080ff808080ffff04ffff01ff33ff3cff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff0effff04ff02ffff04ff09ff80808080ffff02ff0effff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080",
    "solution": "0xffa0a1af73b7dc0f246ffb0bd41b4ac8f0253f4c0949bfdd070d52a3be037767baa3ff8203ffffffff834b65798556616c75658080"
}


### Conditions for Starting Coin

The conditions we need from the starting coin spend must include `(CREATE_COIN SINGLETON_LAUNCHER_HASH)` and `(ASSERT_COIN_ANNOUNCEMENT launcher_announcement)`. These two conditions guarantees that the launcher coin is created and spent in the same spend bundle. An optional condition is the `(CREATE_COIN starting_coin_puzzle_hash)` which returns excess mojos back as a change to the original sender.

In [7]:
from chia.types.condition_opcodes import ConditionOpcode
from chia.util.hash import std_hash
from clvm_tools.binutils import disassemble

launcher_announcement = launcher_solution.get_tree_hash()

# conditions for starting coin
starting_coin_conditions = [
    # create launcher coin with the odd_amount (odd)
    Program.to(
        [
            ConditionOpcode.CREATE_COIN,
            singleton_top_layer.SINGLETON_LAUNCHER_HASH,
            odd_amount,
        ]),
    # assert launcher coin announcement
    Program.to(
        [
            ConditionOpcode.ASSERT_COIN_ANNOUNCEMENT, 
            std_hash(launcher_id + launcher_announcement)
        ]),
    # create change minus fee coin (amount - (odd_amount + fee)) to itself
    Program.to(
        [
            ConditionOpcode.CREATE_COIN,
            starting_puzzle.get_tree_hash(),
            starting_coin.amount - odd_amount,
        ]),
]

for condition in starting_coin_conditions:
    print(disassemble(condition))

(51 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9 1023)
(61 0x14dcea3e86ca6b0b2da5d868ed49c021eab241201fbb2685442a0a43bcfc52a0)
(51 0x4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3 0x01977420d801)


### Spend Starting Coin

A [spend bundle](https://github.com/Chia-Network/chia-blockchain/blob/1.3.5/chia/types/spend_bundle.py) consists of an aggregate signature and one or more [CoinSpend(s)](https://github.com/Chia-Network/chia-blockchain/blob/1.3.5/chia/types/coin_spend.py) which contains, `coin`, `puzzle_reveal`, and `solution`.

The starting coin is a [standard transaction coin](https://chialisp.com/docs/standard_transaction). The coin has a puzzle that pre-commits [DEFAULT_HIDDEN_PUZZLE_HASH](https://github.com/Chia-Network/chiablockchain/blob/1.3.5/chia/wallet/puzzles/p2_delegated_puzzle_or_hidden_puzzle.py#L70) via the curried-in `SYNTHETIC_PUBLIC_KEY`.

```clojure
(SYNTHETIC_PUBLIC_KEY _public_key delegated_puzzle solution)
...

(c
    (list AGG_SIG_ME SYNTHETIC_PUBLIC_KEY (sha256tree1 delegated_puzzle)) 
    (a delegated_puzzle solution)
)
```

#### Solution

In this [Graftroot](https://www.chia.net/2021/05/27/Agrgregated-Sigs-Taproot-Graftroot.html) case, we only need to provide `delegated_puzzle` and `solution` that will give the list of conditions above.

In [8]:
delegated_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_conditions(starting_coin_conditions)
print(disassemble(delegated_puzzle))
solution = p2_delegated_puzzle_or_hidden_puzzle.solution_for_conditions(starting_coin_conditions)
print(disassemble(solution))

(q (51 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9 1023) (61 0x14dcea3e86ca6b0b2da5d868ed49c021eab241201fbb2685442a0a43bcfc52a0) (51 0x4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3 0x01977420d801))
(() (q (51 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9 1023) (61 0x14dcea3e86ca6b0b2da5d868ed49c021eab241201fbb2685442a0a43bcfc52a0) (51 0x4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3 0x01977420d801)) ())


#### AGG_SIG_ME
We also need to provide the signature to satisfy the `AGG_SIG_ME SYNTHETIC_PUBLIC_KEY (sha256tree1 delegated_puzzle)` condition.

In [9]:
from blspy import AugSchemeMPL, PrivateKey
from chia.consensus.default_constants import DEFAULT_CONSTANTS

synthetic_sk: PrivateKey = p2_delegated_puzzle_or_hidden_puzzle.calculate_synthetic_secret_key(
    alice.sk_,
    p2_delegated_puzzle_or_hidden_puzzle.DEFAULT_HIDDEN_PUZZLE_HASH
)
signature = AugSchemeMPL.sign(synthetic_sk,
    (
        delegated_puzzle.get_tree_hash()
        + starting_coin.name()
        + DEFAULT_CONSTANTS.AGG_SIG_ME_ADDITIONAL_DATA
    )
)
starting_coin_spend = CoinSpend(
        starting_coin,
        starting_puzzle,
        solution
)

## Assert the signature (sanity test)
assert AugSchemeMPL.verify(
    p2_delegated_puzzle_or_hidden_puzzle.calculate_synthetic_public_key(
        alice.pk(), 
        p2_delegated_puzzle_or_hidden_puzzle.DEFAULT_HIDDEN_PUZZLE_HASH
    ), 
    delegated_puzzle.get_tree_hash()
        + starting_coin.name()
        + DEFAULT_CONSTANTS.AGG_SIG_ME_ADDITIONAL_DATA, 
    signature
)

print_json(starting_coin_spend.to_json_dict())

{
    "coin": {
        "amount": 1750000000000,
        "parent_coin_info": "0xe3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001",
        "puzzle_hash": "0x4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff0bffff01ff02ffff03ffff09ff05ffff1dff0bffff1effff0bff0bffff02ff06ffff04ff02ffff04ff17ff8080808080808080ffff01ff02ff17ff2f80ffff01ff088080ff0180ffff01ff04ffff04ff04ffff04ff05ffff04ffff02ff06ffff04ff02ffff04ff17ff80808080ff80808080ffff02ff17ff2f808080ff0180ffff04ffff01ff32ff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff06ffff04ff02ffff04ff09ff80808080ffff02ff06ffff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080ffff04ffff01b0a042c855d234578415254b7870b711fb25e8f85beaa4a66bd0673d394c761fa156406c2e3bb375d5b18766d2a12cc918ff018080",
    "solution": "0xff80ffff01ffff33ffa0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9ff8203ff80ffff3dffa014dcea3e86ca6b0b2da5d8

### Create Eve Singleton

Now we have `starting_coin_spend`, `launcher_coin_spend`, and `signature`, we can create the spend bundle and spend it and we should see our first singleton coin on the blockchain.

In [10]:
from chia.types.spend_bundle import SpendBundle

spend_bundle = SpendBundle(
    [starting_coin_spend, launcher_coin_spend],
    signature
)
result = await network.push_tx(spend_bundle)
result

{'additions': [Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: 4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3>, amount=1749999998977),
  Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9>, amount=1023),
  Coin(parent_coin_info=<bytes32: 6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570>, puzzle_hash=<bytes32: a1af73b7dc0f246ffb0bd41b4ac8f0253f4c0949bfdd070d52a3be037767baa3>, amount=1023)],
 'removals': [Coin(parent_coin_info=<bytes32: e3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001>, puzzle_hash=<bytes32: 4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3>, amount=1750000000000),
  Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: eff07522495060c06

In [11]:
eve_singleton_coin_records = await network.sim_client.get_coin_records_by_parent_ids([launcher_id])
print(eve_singleton_coin_records[0].coin)

{'amount': 1023,
 'parent_coin_info': '0x6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570',
 'puzzle_hash': '0xa1af73b7dc0f246ffb0bd41b4ac8f0253f4c0949bfdd070d52a3be037767baa3'}


## Standard Singleton Puzzle
Now we have our first singleton coin. Let's revisit the [singleton top layer puzzle](https://chialisp.com/docs/puzzles/singletons#the-singleton-top-layer) again. The singleton top layer puzzle is an outer puzzle that will guarantee that when it's spent, only one legal singleton coin will be created from it. Other than that restriction, the inner puzzle can do whatever they want.


### Singleton Struct

`SINGLETON_STRUCT` is a collection of three things:
- `SINGLETON_MOD_HASH`
- launcher id
- `SINGLETON_PUZZLE_HASH`

These are helper functions for `SINGLETON_STRUCT`:

[singleton_top_layer.clvm](https://github.com/Chia-Network/chia-blockchain/blob/1.3.5/chia/wallet/puzzles/singleton_top_layer.clvm#L37)
```clojure
  (defun-inline mod_hash_for_singleton_struct (SINGLETON_STRUCT) (f SINGLETON_STRUCT))
  (defun-inline launcher_id_for_singleton_struct (SINGLETON_STRUCT) (f (r SINGLETON_STRUCT)))
  (defun-inline launcher_puzzle_hash_for_singleton_struct (SINGLETON_STRUCT) (r (r SINGLETON_STRUCT)))
```

In [12]:
disassemble(Program.to(singleton_struct))

'(0x24e044101e57b3d8c908b8a38ad57848afd29d3eecc439dba45f4412df4954fd 0x6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570 . 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9)'

In [13]:
%%bash
# mod_hash_for_singleton_struct
brun '(f 1)' '(0x24e044101e57b3d8c908b8a38ad57848afd29d3eecc439dba45f4412df4954fd 0xb5a07c625d08b4dc45d69dcb6291fc9efbfa439432579bbb3dec06f5631d9661 . 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9)'
# launcher_id_for_singleton_struct
brun '(f (r 1))' '(0x24e044101e57b3d8c908b8a38ad57848afd29d3eecc439dba45f4412df4954fd 0xb5a07c625d08b4dc45d69dcb6291fc9efbfa439432579bbb3dec06f5631d9661 . 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9)'
# launcher_puzzle_hash_for_singleton_struct
brun '(r (r 1))' '(0x24e044101e57b3d8c908b8a38ad57848afd29d3eecc439dba45f4412df4954fd 0xb5a07c625d08b4dc45d69dcb6291fc9efbfa439432579bbb3dec06f5631d9661 . 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9)'

0x24e044101e57b3d8c908b8a38ad57848afd29d3eecc439dba45f4412df4954fd
0xb5a07c625d08b4dc45d69dcb6291fc9efbfa439432579bbb3dec06f5631d9661
0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9


### Lineage Proof
> We use this lineage proof to verify that our parent was a singleton. However, in the first spend, the parent is not a singleton and we actually execute a different path where we verify that our parent was a singleton launcher instead.

These are helper functions for **lineage proof**:

[singleton_truths.clib](https://github.com/Chia-Network/chia-blockchain/blob/1.3.5/chia/wallet/puzzles/singleton_truths.clib#L15)
```clojure
  (defun-inline parent_info_for_lineage_proof (lineage_proof) (f lineage_proof))
  (defun-inline puzzle_hash_for_lineage_proof (lineage_proof) (f (r lineage_proof)))
  (defun-inline amount_for_lineage_proof (lineage_proof) (f (r (r lineage_proof))))
  (defun-inline is_not_eve_proof (lineage_proof) (r (r lineage_proof)))
  (defun-inline parent_info_for_eve_proof (lineage_proof) (f lineage_proof))
  (defun-inline amount_for_eve_proof (lineage_proof) (f (r lineage_proof)))
```

In [14]:
eve_proof = singleton_top_layer.lineage_proof_for_coinsol(launcher_coin_spend)
print(eve_proof)
print(disassemble(eve_proof.to_program()))

{'amount': 1023,
 'inner_puzzle_hash': None,
 'parent_name': '0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba'}
(0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba 1023)


In [15]:
%%bash
# is_not_eve_proof
brun '(r (r 1))' '(0x09dd51046893b6d9fd6b96a6cc60faedaf43fc82aad9d2ae817a38608bcba3f5 1023)'
# parent_info_for_eve_proof
brun '(f 1)' '(0x09dd51046893b6d9fd6b96a6cc60faedaf43fc82aad9d2ae817a38608bcba3f5 1023)'
# amount_for_eve_proof
brun '(f (r 1))' '(0x09dd51046893b6d9fd6b96a6cc60faedaf43fc82aad9d2ae817a38608bcba3f5 1023)'

()
0x09dd51046893b6d9fd6b96a6cc60faedaf43fc82aad9d2ae817a38608bcba3f5
1023


In [16]:
%%bash
# calculate launcher id
run '(sha256 0x09dd51046893b6d9fd6b96a6cc60faedaf43fc82aad9d2ae817a38608bcba3f5 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9 1023)'

0xb5a07c625d08b4dc45d69dcb6291fc9efbfa439432579bbb3dec06f5631d9661


In [17]:
%%bash
run calculate_full_puzzle_hash.clsp -i .

(a (q 2 22 (c 2 (c 9 (c 11 (c (a 30 (c 2 (c 5 ()))) ()))))) (c (q ((a . 4) 1 . 1) (a 2 (i 5 (q 2 26 (c 2 (c 13 (c (sha256 18 (sha256 20 24) (sha256 18 (sha256 18 (sha256 20 28) 9) (sha256 18 11 (sha256 20 ())))) ())))) (q . 11)) 1) (sha256 18 (sha256 20 16) (sha256 18 (sha256 18 (sha256 20 28) 5) (sha256 18 (a 26 (c 2 (c 7 (c (sha256 20 20) ())))) (sha256 20 ())))) 2 (i (l 5) (q 11 (q . 2) (a 30 (c 2 (c 9 ()))) (a 30 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 1))


In [18]:
%%bash
# calculate singleton puzzle hash using chialisp
brun '(a (q 2 22 (c 2 (c 9 (c 11 (c (a 30 (c 2 (c 5 ()))) ()))))) (c (q ((a . 4) 1 . 1) (a 2 (i 5 (q 2 26 (c 2 (c 13 (c (sha256 18 (sha256 20 24) (sha256 18 (sha256 18 (sha256 20 28) 9) (sha256 18 11 (sha256 20 ())))) ())))) (q . 11)) 1) (sha256 18 (sha256 20 16) (sha256 18 (sha256 18 (sha256 20 28) 5) (sha256 18 (a 26 (c 2 (c 7 (c (sha256 20 20) ())))) (sha256 20 ())))) 2 (i (l 5) (q 11 (q . 2) (a 30 (c 2 (c 9 ()))) (a 30 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 1))' '((0x24e044101e57b3d8c908b8a38ad57848afd29d3eecc439dba45f4412df4954fd 0xb5a07c625d08b4dc45d69dcb6291fc9efbfa439432579bbb3dec06f5631d9661 . 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9) 0x206318efd67658a08ae16781f0819a216437a7774d98d2af101f8f8d7a03a02e)'

0x0c0d126c89fc434fbf5ddbf6ad2afd1c5529a9df4f099608c6cd536d753948b2


### Singleton Top Layer Diagram

This is the very simplified diagram of how [singleton top layer](https://chialisp.com/docs/puzzles/singletons/#the-singleton-top-layer) works.

<img src="singleton_top_layer.jpg" alt="Boot Strapping Singleton" width="600"/>

In [19]:
# eve singleton coin
eve_singleton_coin_records = await network.sim_client.get_coin_records_by_parent_ids([launcher_id])
eve_singleton_coin = eve_singleton_coin_records[0].coin
print(eve_singleton_coin)
# eve lineage proof
eve_proof = singleton_top_layer.lineage_proof_for_coinsol(launcher_coin_spend)
print(eve_proof)
# puzzle
print(f'singleton top layer puzzle hash: {curried_singleton_puzzle.get_tree_hash()}')
print(f'alice standard txn puzzle hash: {inner_puzzle.get_tree_hash()}')

{'amount': 1023,
 'parent_coin_info': '0x6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570',
 'puzzle_hash': '0xa1af73b7dc0f246ffb0bd41b4ac8f0253f4c0949bfdd070d52a3be037767baa3'}
{'amount': 1023,
 'inner_puzzle_hash': None,
 'parent_name': '0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba'}
singleton top layer puzzle hash: a1af73b7dc0f246ffb0bd41b4ac8f0253f4c0949bfdd070d52a3be037767baa3
alice standard txn puzzle hash: 206318efd67658a08ae16781f0819a216437a7774d98d2af101f8f8d7a03a02e


## Eve Spend

Spending a singleton coin is no different from spending any coin. We just need to provide the proper solution.

- `lineage_proof`: `(0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba 1023)`  
- `my_amount` : 1023 (amount of the coin)
- `inner_solution` : solution for the inner puzzle.

In our case, the inner puzzle is a standard transaction puzzle with `DEFAULT_HIDDEN_PUZZLE_HASH` and alice's pk pre-committed, so the `inner_solution` will be in the standard transaction format below:
`(original_public_key delegated_puzzle solution)`

<img src="singleton_top_layer_and_inner_puzzle.jpg" alt="Boot Strapping Singleton" width="600"/>

We will also need to sign the spend bundle with alice's synthetic secret key as well.

### Eve Spend Conditions

In this singleton spend, two coins will be created. One to create another singleton with the same inner puzzle (alice's standard transaction puzzle) and another one with 1000 mojos to **Bob**. For a new singleton, the amount will be 23 which is still make the singleton valid. We can provide any inner puzzle for a new singleton, but in this case, we just provide the same inner puzzle as the eve singleton coin. The new singleton coin will have the same puzzle hash because it has the same `launcher id`, hence, `SINGLETON_STRUCT` and the same inner puzzle.

In [20]:
bob: Wallet = network.make_wallet("bob")
print(f'bob: {bob.balance()}')
conditions = [
    # Create a new Singleton with the same inner puzzle
    Program.to([
            ConditionOpcode.CREATE_COIN,
            inner_puzzle.get_tree_hash(),
            odd_amount - 1000, # next amount is 23
    ]),
    # Send 1000 mojos to Bob
    Program.to([
            ConditionOpcode.CREATE_COIN,
            bob.puzzle_hash,
            1000,
    ]),
    
]

delegated_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_conditions(conditions)
print(disassemble(delegated_puzzle))
inner_solution = p2_delegated_puzzle_or_hidden_puzzle.solution_for_conditions(conditions)
print(disassemble(inner_solution))
eve_solution = singleton_top_layer.solution_for_singleton(
    eve_proof,
    eve_singleton_coin.amount,
    inner_solution,
)

# signed with alice's synthetic_sk
signature = AugSchemeMPL.sign(synthetic_sk,
    (
        delegated_puzzle.get_tree_hash()
        + eve_singleton_coin.name()
        + DEFAULT_CONSTANTS.AGG_SIG_ME_ADDITIONAL_DATA
    )
)
eve_coin_spend = CoinSpend(
        eve_singleton_coin,
        curried_singleton_puzzle,
        eve_solution)
spend_bundle = SpendBundle(
    [eve_coin_spend],
    signature
)

result = await network.push_tx(spend_bundle)
print(result)
print(f'bob: {bob.balance()}')

bob: 0
(q (51 0x206318efd67658a08ae16781f0819a216437a7774d98d2af101f8f8d7a03a02e 23) (51 0x87908e3f85bf4b55c7e7709915c2ce97a1e6ec1d227e54a04dbfee6862d546a5 1000))
(() (q (51 0x206318efd67658a08ae16781f0819a216437a7774d98d2af101f8f8d7a03a02e 23) (51 0x87908e3f85bf4b55c7e7709915c2ce97a1e6ec1d227e54a04dbfee6862d546a5 1000)) ())
{'additions': [Coin(parent_coin_info=<bytes32: a484f2edad0b69fdf6ef4def3c0e05210a892161688d453d20fb659632ec3a57>, puzzle_hash=<bytes32: 87908e3f85bf4b55c7e7709915c2ce97a1e6ec1d227e54a04dbfee6862d546a5>, amount=1000), Coin(parent_coin_info=<bytes32: a484f2edad0b69fdf6ef4def3c0e05210a892161688d453d20fb659632ec3a57>, puzzle_hash=<bytes32: a1af73b7dc0f246ffb0bd41b4ac8f0253f4c0949bfdd070d52a3be037767baa3>, amount=23)], 'removals': [Coin(parent_coin_info=<bytes32: 6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570>, puzzle_hash=<bytes32: a1af73b7dc0f246ffb0bd41b4ac8f0253f4c0949bfdd070d52a3be037767baa3>, amount=1023)]}
bob: 1000


In [21]:
coin_records = await network.sim_client.get_coin_records_by_parent_ids([eve_singleton_coin.name()])
singleton_coin_2 = next(cr for cr in coin_records if cr.coin.amount%2 != 0).coin
print(singleton_coin_2)

{'amount': 23,
 'parent_coin_info': '0xa484f2edad0b69fdf6ef4def3c0e05210a892161688d453d20fb659632ec3a57',
 'puzzle_hash': '0xa1af73b7dc0f246ffb0bd41b4ac8f0253f4c0949bfdd070d52a3be037767baa3'}


In [22]:
lineage_proof = singleton_top_layer.lineage_proof_for_coinsol(eve_coin_spend)
print(f'lineage_proof: {lineage_proof}')
conditions = [
    # terminate singleton
    Program.to([
            ConditionOpcode.CREATE_COIN,
            inner_puzzle.get_tree_hash(),
            -113, # terminate singleton
    ]),
    # Send 22 mojos to Bob
    Program.to([
            ConditionOpcode.CREATE_COIN,
           bob.puzzle_hash,
            22, # Can't be odd amount
    ]),
    
]

delegated_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_conditions(conditions)
print(disassemble(delegated_puzzle))
inner_solution = p2_delegated_puzzle_or_hidden_puzzle.solution_for_conditions(conditions)
print(disassemble(inner_solution))
solution = singleton_top_layer.solution_for_singleton(
    lineage_proof,
    singleton_coin_2.amount,
    inner_solution,
)
print(disassemble(solution))

# signed with alice's synthetic_sk
signature = AugSchemeMPL.sign(synthetic_sk,
    (
        delegated_puzzle.get_tree_hash()
        + singleton_coin_2.name()
        + DEFAULT_CONSTANTS.AGG_SIG_ME_ADDITIONAL_DATA
    )
)

coin_spend = CoinSpend(
        singleton_coin_2,
        curried_singleton_puzzle,
        solution)
    
spend_bundle = SpendBundle(
    [coin_spend],
    signature
)

result = await network.push_tx(spend_bundle)
print(result)
print(f'bob: {bob.balance()}')

lineage_proof: {'amount': 1023,
 'inner_puzzle_hash': '0x206318efd67658a08ae16781f0819a216437a7774d98d2af101f8f8d7a03a02e',
 'parent_name': '0x6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570'}
(q (51 0x206318efd67658a08ae16781f0819a216437a7774d98d2af101f8f8d7a03a02e -113) (51 0x87908e3f85bf4b55c7e7709915c2ce97a1e6ec1d227e54a04dbfee6862d546a5 22))
(() (q (51 0x206318efd67658a08ae16781f0819a216437a7774d98d2af101f8f8d7a03a02e -113) (51 0x87908e3f85bf4b55c7e7709915c2ce97a1e6ec1d227e54a04dbfee6862d546a5 22)) ())
((0x6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570 0x206318efd67658a08ae16781f0819a216437a7774d98d2af101f8f8d7a03a02e 1023) 23 (() (q (51 0x206318efd67658a08ae16781f0819a216437a7774d98d2af101f8f8d7a03a02e -113) (51 0x87908e3f85bf4b55c7e7709915c2ce97a1e6ec1d227e54a04dbfee6862d546a5 22)) ()))
{'additions': [Coin(parent_coin_info=<bytes32: c4f8bec22989b15ef07ab5ebe00b082a77f5abe68e7d5869380e4a753efb9565>, puzzle_hash=<bytes32: 87908e3f85bf4b55c7e77099

## Use Cases

### Plot NFT for Pooling
The most well-known singleton example is the [plot nft](https://github.com/Chia-Network/chia-blockchain/wiki/Pooling-FAQ#what-is-a-plot-nft), which keeps the pool that the farmer belongs to. Farmer can always switch the pool by spending the singleton plot nfg and provide the new pool information. 

> The original pool design was having pool keys in plot files, but that means if farmers want to switch pool, they'd have to re-plot!! [Pools in Chia](https://www.chia.net/2020/11/10/pools-in-chia.html)

### Others

Anything that needs to be unique on the blockchain can also be represented by a singleton.

- NFT (Non-Fungible Token) 
- DIDs (Decentralized Identifiers) : A parent singleton can be treated as DID.

## Conclusion

In this notebook, we show how we can create a singleton from a launcher coin, spend it, and terminate it. Like the standard transaction puzzle, the singleton top layer puzzle is very powerful and customizable.

# References

- [What is a singleton?](https://developers.chia.net/t/what-is-a-singleton/87)
- [Can you store state on the network?](https://developers.chia.net/t/can-you-store-state-on-the-network/84)
- [Singletons | Chialisp Docs](https://chialisp.com/docs/puzzles/singletons)
- [Singletons and Ethereum-like Contracts in Chia | Chialisp](https://www.youtube.com/watch?v=kA0l9n5SEI8)
- [Official Pooling Protocol Launched](https://www.chia.net/2021/07/07/official-pooling-launched.en.html)
- [Using the EVM to simplify Chia Pools](https://medium.com/liquidum-network/using-the-evm-to-simplify-chia-pools-d5bd696411c3)
- [New Singleton 1.1 Standard Top Layer](https://developers.chia.net/t/new-singleton-1-1-standard-top-layer/387)
- [singleton_top_layer.py](https://github.com/Chia-Network/chia-blockchain/blob/optimize_singleton_v1.1/chia/wallet/puzzles/singleton_top_layer.py)
- [`logand`](https://chialisp.com/docs/ref/clvm#bit-operations)